## <i> <span style='color:red'>SVM-SALARY DATA</i></span>

### PROBLEM STATMENT: Prepare a classification model using SVM for salary data 

### Import libraries

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler

from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings("ignore")

### Reading test and train data

In [2]:
df_train = pd.read_csv('SalaryData_Train(1).csv')
df_train.head()

age          workclass   education  educationno        maritalstatus  \
0   39          State-gov   Bachelors           13        Never-married   
1   50   Self-emp-not-inc   Bachelors           13   Married-civ-spouse   
2   38            Private     HS-grad            9             Divorced   
3   53            Private        11th            7   Married-civ-spouse   
4   28            Private   Bachelors           13   Married-civ-spouse   

           occupation    relationship    race      sex  capitalgain  \
0        Adm-clerical   Not-in-family   White     Male         2174   
1     Exec-managerial         Husband   White     Male            0   
2   Handlers-cleaners   Not-in-family   White     Male            0   
3   Handlers-cleaners         Husband   Black     Male            0   
4      Prof-specialty            Wife   Black   Female            0   

   capitalloss  hoursperweek          native  Salary  
0            0            40   United-States   <=50K  
1            0            13   United-States   <=50K  
2            0            40   United-States   <=50K  
3            0            40   United-States   <=50K  
4            0            40            Cuba   <=50K

In [3]:
df_test = pd.read_csv('SalaryData_Test(1).csv')
df_test.head()

age   workclass      education  educationno        maritalstatus  \
0   25     Private           11th            7        Never-married   
1   38     Private        HS-grad            9   Married-civ-spouse   
2   28   Local-gov     Assoc-acdm           12   Married-civ-spouse   
3   44     Private   Some-college           10   Married-civ-spouse   
4   34     Private           10th            6        Never-married   

           occupation    relationship    race    sex  capitalgain  \
0   Machine-op-inspct       Own-child   Black   Male            0   
1     Farming-fishing         Husband   White   Male            0   
2     Protective-serv         Husband   White   Male            0   
3   Machine-op-inspct         Husband   Black   Male         7688   
4       Other-service   Not-in-family   White   Male            0   

   capitalloss  hoursperweek          native  Salary  
0            0            40   United-States   <=50K  
1            0            50   United-States   <=50K  
2            0            40   United-States    >50K  
3            0            40   United-States    >50K  
4            0            30   United-States   <=50K

### Checking the basic info about the data

In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15060 entries, 0 to 15059
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            15060 non-null  int64 
 1   workclass      15060 non-null  object
 2   education      15060 non-null  object
 3   educationno    15060 non-null  int64 
 4   maritalstatus  15060 non-null  object
 5   occupation     15060 non-null  object
 6   relationship   15060 non-null  object
 7   race           15060 non-null  object
 8   sex            15060 non-null  object
 9   capitalgain    15060 non-null  int64 
 10  capitalloss    15060 non-null  int64 
 11  hoursperweek   15060 non-null  int64 
 12  native         15060 non-null  object
 13  Salary         15060 non-null  object
dtypes: int64(5), object(9)
memory usage: 1.6+ MB


In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30161 entries, 0 to 30160
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            30161 non-null  int64 
 1   workclass      30161 non-null  object
 2   education      30161 non-null  object
 3   educationno    30161 non-null  int64 
 4   maritalstatus  30161 non-null  object
 5   occupation     30161 non-null  object
 6   relationship   30161 non-null  object
 7   race           30161 non-null  object
 8   sex            30161 non-null  object
 9   capitalgain    30161 non-null  int64 
 10  capitalloss    30161 non-null  int64 
 11  hoursperweek   30161 non-null  int64 
 12  native         30161 non-null  object
 13  Salary         30161 non-null  object
dtypes: int64(5), object(9)
memory usage: 3.2+ MB


### Label encoding-train data

In [6]:
df_train = df_train.apply(LabelEncoder().fit_transform)
df_train.head()

age  workclass  education  educationno  maritalstatus  occupation  \
0    8          2          1            6              4           6   
1   21          2         11            8              2           4   
2   11          1          7           11              2          10   
3   27          2         15            9              2           6   
4   17          2          0            5              4           7   

   relationship  race  sex  capitalgain  capitalloss  hoursperweek  native  \
0             3     2    1            0            0            39      37   
1             0     4    1            0            0            49      37   
2             0     4    1            0            0            39      37   
3             0     2    1           87            0            39      37   
4             1     4    1            0            0            29      37   

   Salary  
0       0  
1       0  
2       1  
3       1  
4       0

### Label encoding- Test data

In [7]:
df_test = df_test.apply(LabelEncoder().fit_transform)
df_test.head()

age  workclass  education  educationno  maritalstatus  occupation  \
0   22          5          9           12              4           0   
1   33          4          9           12              2           3   
2   21          2         11            8              0           5   
3   36          2          1            6              2           5   
4   11          2          9           12              2           9   

   relationship  race  sex  capitalgain  capitalloss  hoursperweek  native  \
0             1     4    1           24            0            39      37   
1             0     4    1            0            0            12      37   
2             1     4    1            0            0            39      37   
3             0     2    1            0            0            39      37   
4             5     2    0            0            0            39       4   

   Salary  
0       0  
1       0  
2       0  
3       0  
4       0

### Splitting target variable and independent variable-Train data

In [8]:
X_train= df_train.drop(['education','relationship','native','maritalstatus','sex','race','Salary'],axis=1).values
Y_train= df_train['Salary'].values
print(np.unique(Y_train))
X_train

[0 1]


array([[ 8,  2,  6, ...,  0,  0, 39],
       [21,  2,  8, ...,  0,  0, 49],
       [11,  1, 11, ...,  0,  0, 39],
       ...,
       [21,  2, 12, ...,  0,  0, 49],
       [27,  2, 12, ..., 73,  0, 39],
       [18,  3, 12, ...,  0,  0, 59]], dtype=int64)

### Splitting target variable and independent variable- test data

In [9]:
X_test= df_test.drop(['education','relationship','native','maritalstatus','sex','race','Salary'],axis=1).values
Y_test= df_test['Salary'].values
print(np.unique(Y_test))
X_test

[0 1]


array([[ 22,   5,  12, ...,  24,   0,  39],
       [ 33,   4,  12, ...,   0,   0,  12],
       [ 21,   2,   8, ...,   0,   0,  39],
       ...,
       [ 41,   2,   8, ...,   0,   0,  39],
       [  5,   2,   8, ...,   0,   0,  19],
       [ 35,   3,   8, ..., 107,   0,  39]], dtype=int64)

## SVM
### 1) Linear support vector machine,  gamma=0.22 
#### checking the accuracy score

In [10]:
svc = SVC(gamma=0.22)
svc.fit(X_train, Y_train)
score_svc = svc.score(X_test,Y_test)
print('The accuracy of SVC is', score_svc)

The accuracy of SVC is 0.7630715161964126


### 2)Radial support vector machine  , gamma=0.22
#### checking the accuracy score

In [11]:
svc_radical =svm.SVC(kernel='rbf',C=1,gamma=0.22)
svc_radical.fit(X_train,Y_train)
score_svc_radical = svc_radical.score(X_test,Y_test)
print('The accuracy of Radical SVC Model is', score_svc_radical)

The accuracy of Radical SVC Model is 0.7630715161964126


### 3) Sigmoid support vector machine
#### checking the accuracy score

In [12]:
svc_radical_sig =svm.SVC(kernel='sigmoid',C=1,gamma=0.22)
svc_radical_sig.fit(X_train,Y_train)
score_svc_radical_sig = svc_radical_sig.score(X_test,Y_test)
print('The accuracy of sigmoid SVC Model is', score_svc_radical_sig)

The accuracy of sigmoid SVC Model is 0.7510692616292564


### Checking the svm with another gamma value
### 1) linear svm

In [13]:
svc_1 = SVC(gamma=0.33)
svc_1.fit(X_train, Y_train)
score_svc_1 = svc_1.score(X_test,Y_test)
print('The accuracy of SVC is', score_svc_2)

The accuracy of SVC is 0.7594244222671662


### constructing svm with kernel=polynomial

In [ ]:
svc_poly =svm.SVC(kernel='poly',C=1,gamma=0.33)
svc_poly.fit(X_train,Y_train)
score_svc_poly = svc_poly.score(X_test,Y_test)
print('The accuracy of poly SVC Model is', score_svc_poly) 

### constructing svm with kernel=sigmoid

In [ ]:
svc_sig =svm.SVC(kernel='sigmoid',C=1,gamma=0.33)
svc_sig.fit(X_train,Y_train)
score_svc_sig = svc_sig.score(X_test,Y_test)
print('The accuracy of poly SVC Model is', score_svc_sig) 

### Tabling all the models with their accuracy score

In [ ]:
t={'model':['Lin_SVM_gamma_0.22','Radial_SVM','Sigmoid','Lin_SVM_gamma_0.33','Poly_SVM','Sigmoid_SVM'],
  'Acc_score':[score_svc,score_svc_radical,score_svc_radical_sig,score_svc_1,score_svc_poly,score_svc_sig]
  }
t=pd.DataFrame(t)
t

###  Predicting the model in test data with kernel=polynomial (which having high accuracy score)


In [ ]:
clf = SVC(kernel='poly',C=1,gamma=0.33)
clf.fit(X_train , Y_train)
y_pred = clf.predict(X_test)


###  confusion matrix for accuracy score of test data

In [ ]:
acc = accuracy_score(Y_test, y_pred) * 100
print("Accuracy =", acc)
confusion_matrix(Y_test, y_pred)

### Classification report for test data

In [ ]:
print(classification_report(Y_test, y_pred))

### visualization using histogram 

In [ ]:
plt.rcParams['figure.figsize'] = (9,9) 
_, ax = plt.subplots()
ax.hist(Y_test, color = 'r', alpha = 0.5, label = 'actual', bins=3)
ax.hist(y_pred, color = 'c', alpha = 0.5, label = 'prediction', bins=3)
ax.yaxis.set_ticks(np.arange(0,11))
ax.legend(loc = 'best')
plt.show()

## Inference:

### From the accuracy score its clear that polynomial svm is having the best score among them.So the best classification model in svm for salary data is polynomial with highest accuracy score 99.98